# Exemple de code utilisant PyTorch

In [1]:
#!pip install torch torchvision

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
# 1. Configuration de base
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 64
EPOCHS = 5
LR = 0.001

In [4]:
# 2. Préparation des données
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

100%|██████████| 9.91M/9.91M [00:01<00:00, 9.71MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 377kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 2.89MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.79MB/s]


In [5]:
# 3. Définition du modèle
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
    
    def forward(self, x):
        return self.net(x)

model = SimpleMLP().to(DEVICE)

In [6]:
# 4. Optimiseur et fonction de perte
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [7]:
# 5. Entraînement
def train():
    model.train()
    for epoch in range(EPOCHS):
        total_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Époch {epoch+1}/{EPOCHS} — Perte moyenne: {total_loss / len(train_loader):.4f}")

In [8]:
# 6. Évaluation sur le set de test
def test():
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)
    print(f"Précision sur test : {100 * correct / total:.2f}%")

In [9]:
# 7. Sauvegarde / Chargement
def save_model(path="mnist_model.pth"):
    torch.save(model.state_dict(), path)

def load_model(path="mnist_model.pth"):
    model.load_state_dict(torch.load(path, map_location=DEVICE))
    model.to(DEVICE)


if __name__ == "__main__":
    train()
    test()
    #save_model()
    print("Modèle sauvegardé.")


Époch 1/5 — Perte moyenne: 0.3448
Époch 2/5 — Perte moyenne: 0.1567
Époch 3/5 — Perte moyenne: 0.1085
Époch 4/5 — Perte moyenne: 0.0831
Époch 5/5 — Perte moyenne: 0.0654
Précision sur test : 97.56%
Modèle sauvegardé.
